# Потоки

Для начала, вспомним, как организовано выполнение программ на ПК. Программа представляет из себя набор инструкций, которые выполняются процессором. Процессор (CPU) состоит из устройства управления (Control Unit) и арифметико-логического устройства (Arithmetic Logic Unit). Устройство управления получает инструкции из памяти, декодирует их и передает на выполнение в арифметико-логическое устройство. После выполнения инструкции, управление возвращается в устройство управления, которое получает следующую инструкцию и передает ее на выполнение. Таким образом, процессор последовательно выполняет инструкции, которые составляют программу.

![Процессор](./images/processor.jpeg)

Как видно, процессор может выполнить одну инструкцию за раз. Однако, если мы посмотрим загрузку процессора типичного ПК, то увидим, что он не работает на 100%. При этом в системе выполняется множество программ. 

Как это возможно? Дело в том, что процессор не выполняет инструкции программы последовательно, а переключается между ними. При этом, если одна из программ заблокирована, то процессор переключается на другую программу, которая может быть выполнена. Таким образом, процессор выполняет инструкции программы не последовательно, а параллельно. 

При этом, если в системе несколько процессоров, то они могут выполнять инструкции разных программ одновременно. Такая организация выполнения программ называется многозадачностью (multitasking).

Для реализации многозадачности в операционной системе существует понятие потока (**thread**). Поток — это основная единица, которой операционная система выделяет время процессора. Упрощенно, поток представляет собой некоторую последовательность инструкций, которая выполняется "параллельно" с другими. При этом, все потоки в системе выполняются одновременно, но на самом деле процессор последовательно выполняет инструкции каждого потока. Таким образом, потоки позволяют реализовать параллельное выполнение программ.

Для разделения времени процессора между потоками, операционная система использует понятие кванта времени (**time slice**). Квант времени — это некоторый промежуток времени, в течение которого процессор выполняет инструкции одного потока. По истечении кванта времени, процессор переключается на другой поток. Таким образом, операционная система реализует многозадачность.

![Потоки](./images/threads.png)

Для разделения программ между потоками, операционная система использует понятие процесса (**process**). Процесс — это некоторая программа, которая выполняется в системе. Однако, процесс не является физическим объектом, а является абстракцией, которая представляет из себя набор ресурсов, необходимых для выполнения программы. Каждый процесс может состоять из нескольких потоков.

Различные процессы в системе выполняются независимо друг от друга. При этом, каждый процесс имеет свое адресное пространство, которое изолировано от других процессов. Таким образом, процессы не могут обращаться к памяти друг друга.

Различные технологии и фреймворки предоставляют различные способы работы с потоками. Давайте рассмотрим инструменты, которые предоставляет dotnet.

In [ ]:
using System.Threading;

static int Main(string[] args)
{
    Thread t = new Thread (new ThreadStart(WriteY)); // Kick off a new thread
    t.Start();                                      // running WriteY()

    // Simultaneously, do something on the main thread.
    for (int i = 0; i < 100; i++)
    {
        Console.Write ("x");
    }
    
    return 0;
}

static void WriteY()
{
    for (int i = 0; i < 100; i++) Console.Write ("y");
}

Main(new string[0]);

Для создания потока в dotnet используется класс `Thread` пространства имен `System.Threading`. Единицей исполнения в потоке является делегат (метод), который передается в конструктор. При этом, выполнение инструкций в потоке начнется только после вызова метода `Start()`.

При запуске консольного приложения, в системе создается процесс, который выполняет код в методе `Main()`. При этом поток, выполняющий метод `Main()` называется основным потоком (Main Thread) . При вызове метода `Start()` инструкции, определенные в методе `WriteY()` таже начинают "параллельно" исполняться.

Процесс, запущенный при старте приложения dotnet (host process) будет изолирован от других процессов на уровне системы. Потоки же, запущенные внутри процесса, будут разделять ресурсы процесса. И в случае с dotnet, степень их изоляции будет ограничена контекстом выполнения делегата, а это в первую очередь стек вызовов и локальные переменные.

Кроме этого, в контексте потока сохраняется текущие культурные настройки, приоритет планирования потока и другие параметры. Таким образом мы можем говорить, что программа dotnet имеет столько стеков вызовов, сколько потоков в ней выполняется.

Рассмотрим на примере как это работает

In [3]:
using System.Threading;

static int Main(string[] args)
{
    Thread t = new Thread (new ParameterizedThreadStart(Write)); // Kick off a new thread
    t.Start("y");                               // running WriteY()

    Write("x");                       // Simultaneously, do something on the main thread.
    return 0;
}

static void Write(object y)
{
    for (int i = 0; i < 100; i++)
    {
        Console.Write($"{i}:{y}");
    }
}

Main(new string[0]);
        

0:x1:x2:x3:x4:x5:x0:y1:y2:y3:y4:y5:y6:y7:y8:y9:y10:y11:y12:y13:y14:y15:y16:y17:y18:y19:y20:y21:y22:y6:x7:x8:x9:x10:x23:y24:y25:y26:y27:y28:y11:x12:x13:x14:x15:x16:x17:x18:x19:x20:x21:x22:x23:x24:x25:x26:x27:x28:x29:x30:x31:x29:y30:y31:y32:y33:y34:y35:y36:y37:y38:y39:y40:y41:y42:y43:y44:y45:y46:y47:y48:y49:y50:y51:y52:y53:y54:y55:y56:y57:y58:y59:y60:y61:y62:y63:y64:y65:y66:y67:y68:y69:y70:y71:y72:y73:y74:y75:y76:y77:y78:y79:y80:y81:y82:y83:y84:y85:y86:y87:y88:y89:y90:y91:y92:y93:y94:y95:y96:y97:y98:y99:y32:x33:x34:x35:x36:x37:x38:x39:x40:x41:x42:x43:x44:x45:x46:x47:x48:x49:x50:x51:x52:x53:x54:x55:x56:x57:x58:x59:x60:x61:x62:x63:x64:x65:x66:x67:x68:x69:x70:x71:x72:x73:x74:x75:x76:x77:x78:x79:x80:x81:x82:x83:x84:x85:x86:x87:x88:x89:x90:x91:x92:x93:x94:x95:x96:x97:x98:x99:x

В этом примере поток создается с делегатом, который при вызове будет принимать параметр типа `object`, который является параметром запуска потока, и передается в метод `Start()`.

Поскольку значения параметров и переменных хрпняться на стеке, то все значения параметров и локальных переменных будут разными для каждого потока. В отличие от значений, хранящихся на стеке, значения, хранящиеся в куче, будут общими для всех потоков.

Такая особенность показывает, почему многопоточность полезна: один поток может получать данные в фоновом режиме, например,
в то время как другой поток может отображать данные по мере их поступления.

In [13]:
public class Program
{
    bool done;

    // Note that Go is now an instance method
    public void Go()
    {
        if (!done)
        { 
            done = true;
            Console.WriteLine("Done");
        }
    }
}

static int Main()
{
    Program tt = new Program();   // Create a common instance
    new Thread(tt.Go).Start();
    tt.Go();

    return 0;
}

Main();

Done


Статические данные (значений полей) как и статические члены типов являются общими для всех потоков. Используем это свойство, однако немного поменяем  порядок вывода сообщений в консоль.

In [29]:

static bool done;    // Static fields are shared between all threads

static int Main()
{
    new Thread(Go).Start();
    Go();

    return 0;
}

static void Go()
{
    if (!done)
    { 
        Console.WriteLine("Done");
        done = true; 
    }
}

Main();

Done
Done


Как видно, в этом случае, сообщение может быть выведено более одного раза. Это связано с тем, что в одном потоке, значение `done` может быть изменено после того, как другой поток проверит его значение. Такая ситуация называется **race condition** (состояние гонки) или просто конкуррентным доступом к ресурсам. 

Это классическая проблема многопоточности, которая может возникнуть, когда два или более потока имеют доступ к общему ресурсу. В этом случае, общий ресурс - это поле `done`. Для ее решения необходимо использовать механизм синхронизации доступа к ресурсу.

Один из таких механизмов применим в следующем примере

In [ ]:

static bool done;
static readonly object locker = new object();

static void Main()
{
    new Thread(Go).Start();
    Go();
}

static void Go()
{
    lock (locker)
    {
        if (!done)
        {
            Console.WriteLine("Done"); done = true;
        }
    }
}

С помощью ключевого слова `lock` мы можем обеспечить выполнение определенного блока кода только одним потоком (по очереди). Это обеспечивается с помощью взаимоисключающей (mutually exclusive) блокировки общего ресурса. В данном случае, ресурсом является объект `locker`. В этом случае, если один поток уже захватил объект `locker`, то другой поток будет ждать, пока первый поток не освободит его. Таким образом, мы можем гарантировать, что в блоке `lock` будет выполняться только один поток.

Но почему мы не используем само поле `done` в качестве ресурса? Дело в том что сама контрукция является короткой записью для следующего кода

In [ ]:

static bool done;
static readonly object locker = new object();

static void Main()
{
    new Thread(Go).Start();
    Go();
}

static void Go()
{
    bool lockWasTaken = false;
    var temp = locker;
    try
    {
        Monitor.Enter(temp, ref lockWasTaken);
        {
            if (!done)
            {
                 Console.WriteLine("Done");
                 done = true;
            }
        }
    }
    finally
    {
        if (lockWasTaken)
        {
            Monitor.Exit(temp);
        }
    }
}

Этот механизм называется **монитором** (monitor) и требует в качестве ресурса объект ссылочного типа. При этом, в случае, если бы мы передали в качестве ресурса поле `done`, монитром был бы захвачен объект, который бы создался с помощью механизма автоматического упаковывания (boxing) и который был бы разным для каждого из потоков.

Монитор - это внутренний механизм dotnet, и работает благодаря внутреннему способу хранения объектов, в котором каждый объект ссылочного типа имеет внутри себя индекс специальной таблицы, в которой хранятся все объекты, которые используются в качестве ресурса монитора.

![Монитор](./images/object.gif)

Упрощенно, процесс доступа к монитору можно представить следующим образом

![Монитор](./images/monitor.webp)

Еще одной особенностью этого механизма является то, что если один поток захватил монитор, то он может повторно захватить его, не вызывая блокировки. Это позволяет избежать блокировки в случае, если внутри блока `lock` вызывается метод, который сам использует блокировку. Например, следующий код не вызовет блокировку.

Кроме того, только тот поток, который захватил монитор, может его освободить. Поэтому, невозможно вызвать метод `Monitor.Exit` в другом потоке, если он не захватил ресурс.

Чтобы стало еще интереснее, давайте рассмотрим следующий пример

In [ ]:

static bool done;
static object locker1 = new object();
static object locker2 = new object();

static int Main()
{
    new Thread(() =>
    {
        lock (locker1)
        {
            Thread.Sleep(1000);
            lock (locker2) // Deadlock
            {
                if (!done) { Console.WriteLine("Done"); done = true; }
            }
        }
    }).Start();

    lock (locker2)
    {
        Thread.Sleep(1000); // Deadlock
        {
            if (!done) { Console.WriteLine("Done"); done = true; }
        }
    }

    return 0;
}

Main();

Давайте рассмотрим пример поподробнее. Первый поток захватывает ресурс `locker1`, затем он засыпает на 1 секунду. Второй поток захватывает монитор `locker2` и засыпает на 1 секунду. После этого, первый поток пытается захватить ресурс `locker2`, но он уже захвачен вторым потоком. Второй поток пытается захватить ресурс `locker1`, но он уже захвачен первым потоком. Таким образом, оба потока ожидают освобождения ресурс, но никто не может их освободить, так как каждый поток захватил ресурс, который нужен другому потоку. 

Такая ситуация называется **взаимной блокировкой** (deadlock). Программа при этом не завершается (зависает), так как оба потока находятся в состоянии ожидания.

Тут следует сделать замечание о состояниях потоков. Поток может находиться в одном из следующих основных состояний (все возможные состояния можно найти в перечислении `ThreadState` и по ссылке https://docs.microsoft.com/en-us/dotnet/api/system.threading.threadstate?view=net-7.0).
 
 - `Aborted` Поток не выполняет работу, но его состояние еще не изменилось на Stopped.
 - `Running` Поток был запущен и выполняется в данный момент.
 - `Stopped` Поток был остановлен.
 - `Suspended` Поток был приостановлен.
 - `Unstarted` Метод *Start()* не был вызван для потока.
 - `WaitSleepJoin` Поток заблокирован. Это может произойти в результате вызова метода *Sleep(Int32)* или метода *Join()*, в результате запроса блокировки, например при вызове метода *Enter(Object)* или *Wait(Object, Int32, Boolean)* или в результате ожидания объекта синхронизации потока, такого как *ManualResetEvent*.

Текущее состояние потока можно получить с помощью свойства `Thread.ThreadState`. В нашем примере, оба потока находятся в состоянии `WaitSleepJoin` и не выполняют никакой работы. Состояние потока кроме методов, которые мы использовали, могут изменить методы `Abort()`, `Suspend()`, `Join()`. Давайте рассмотрим их подробнее.

Для начала, давайте просто выполним метод `Go` на одном потоке. 

In [ ]:

static int Main()
{
    Thread t = new Thread(Go);
    t.Start();
    Console.WriteLine("Thread t has ended!");
    return 0;
}

static void Go()
{
    for (int i = 0; i < 10; i++)
    {
        Console.Write("y");
    }
}

Main();

Тут видно, что сообщение `Thread t has ended!` появилось во время выполнения метода `Go`, но как дождаться завершения выполнения этого метода? Тут поможет метод `Join()` у потока `t`.

In [ ]:
static int Main()
{
    Thread t = new Thread(Go);
    t.Start();
    Console.WriteLine("Thread t has ended!");
    t.Abort();
    return 0;
}

static void Go()
{
    for (int i = 0; i < 10; i++)
    {
        Console.Write("y");
    }
}

Main();